In [ ]:
!pip 

In [21]:
import cv2
from tensorflow.python.keras.applications.densenet import preprocess_input
from tensorflow.python.keras.backend import set_session
from tensorflow.python.keras.models import Model
from tensorflow.python.keras.preprocessing import image
import matplotlib.pyplot as plt
import numpy as np
import pickle
from random import shuffle
from scipy.misc import imread
from scipy.misc import imresize
import tensorflow as tf

from model.model import SSD
# from ssd_training import MultiboxLoss
# from ssd_utils import BBoxUtility
#Make objects of classes
import ssd_training
import ssd_utils

# from tensorflow.python.keras.applications import imagenet_utils

# from tensorflow.python.keras.layers import Activation
# from tensorflow.python.keras.layers import Input

%matplotlib inline
plt.rcParams['figure.figsize'] = (8, 8)
plt.rcParams['image.interpolation'] = 'nearest'
np.set_printoptions(suppress=True)

In [22]:
NUM_CLASSES = 4
input_shape = (300, 300, 3)

In [23]:
priors = pickle.load(open('voc_2007.pkl', 'rb'))
keys = sorted(gt.keys())
num_train = int(round(0.8 * len(keys)))
train_keys = keys[:num_train]
val_keys = keys[num_train:]
num_val = len(val_keys)

FileNotFoundError: [Errno 2] No such file or directory: 'voc_2007.pkl'